In [6]:
import torch

from ssm_models import SelectiveScanModelLooped
from models import TransformerModelLooped
from curriculum import CurriculumSimple
from train import train_without_config, validate_model

In [2]:
n_dims = 10

In [7]:
model_ssm_b5 = TransformerModelLooped(n_dims=n_dims,
                                 n_positions=101,
                                 n_embd=256,
                                 n_layer=1,
                                 n_head=4,
                                 pred_type="regression").cuda()

cirriculum_ssm_b5 = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [1000, 5, 1])

metrics_l1_b5_d10 = train_without_config(
    model_ssm_b5, cirriculum_ssm_b5, model_n_dims=n_dims,
    log_every_steps=10, train_steps=15000, family="ssm_gpt2_loop",
    do_wandb_log=False, seed=None, task_name="linear_regression")

In [5]:
from scripts.tasks import get_task_sampler

model = SelectiveScanModelLooped(n_dims=10,
                                 n_positions=101,
                                 n_embd=256,
                                 n_layer=1,
                                 n_head=4,
                                 pred_type="regression").cuda()

task_sampler = get_task_sampler(
    task_name="linear_regression",
    batch_size=64,
    n_points=31,
    n_dims=10,
    n_dims_truncated=10,
    device="cuda",
)

real_task = task_sampler()
xs, ys = real_task.xs.float(), real_task.ys.float()

horizon_start = max(0, 5 - 20)
model(xs, ys, horizon_start, 5)